## import 

In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
import pytest
from ROOT import TFile, gSystem
output_notebook()
output_file("performanceMap.html")

## Load trees for comparison

In [ ]:
treeAlphaV, treeListAlphaV, fileListAlphaV = LoadTrees("cat performance.list",".*_alphaVDist","xxx","",0)
# treeListAlphaV                           # enable this to get tree list
# treeAlphaV.GetListOfFriends().ls()       # enable this to get tree list

## Select branches/aliases example:

In [ ]:
findSelectedBranches(treeAlphaV,[".*Center"],exclude=[".*his.*"])
findSelectedBranches(treeAlphaV,[".*TRD.*CovarP1.*meanG"],exclude=[".*XXX.*"])

## Create data frame example

In [ ]:
#tree2Panda?

In [ ]:
selection="entries>100&&abs(tglCenter)<0.35"
dfVariables=tree2Panda(treeAlphaV,[".*Center",".*entries"],selection, exclude=[".*his.*"])
dfCovar=tree2Panda(treeAlphaV,[".*TRD.*CovarP4.*binMedian"],selection,exclude=[".*ITS_.*"],
                  columnMask=[["_qPt_tgl_alphaVDist_binMedian",""], ["_hisCovar","_Cov"]])
dfCovar = pd.concat([dfCovar,dfVariables],axis=1)
print(dfCovar.columns)
dfCovar["sentries"]=dfCovar.eval("sqrt(1/entries)")
dfCovar["sector"]=dfCovar.eval("9*alphaVCenter/3.1497")
display(dfCovar.head(5))
#dfCovar = dfCovar.sample(4000)

In [ ]:
output_file("performanceMap_covarP4_alpha.html")
figureArray = [
    #[['alphaVCenter'], ['TRDOff_CovP4_TRDv'], {"size": 4, "colorZvar":"qPtCenter"}],
    #[['alphaVCenter'], ['TRDOn_CovP4_TRDv'], {"size": 4, "colorZvar":"qPtCenter"}],
    [['sector'], ['TRDOff_CovP4_TRDv','TRDOn_CovP4_TRDv'], {"size": 4}],
    [['sector'], ['TRDOn_CovP4_TRDv/TRDOff_CovP4_TRDv'], {"color":"black","size": 4}],
    ['table']
]

widgetParams=[
    ['range', ['sector']],
    #['range', ['qPtCenter']],
    #['range', ['tglCenter']],
    ['select',["qPtCenter"]],
    ['select',["tglCenter"]],
    ['range', ['sentries']],
]
tooltips = [("qPt", "@qPtCenter 1/GeV"), ("tgl", "@tglCenter"), ("entries", "@entries")]
widgetLayoutDesc=[ [0,1], [2,3], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0],
    [1,{'plot_height':100}],
    {'plot_height':200,'sizing_mode':'scale_width','commonX':1}
]
fig=bokehDrawSA.fromArray(dfCovar, "abs(tglCenter)<0.4", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc)

In [ ]:
#treeAlphaV.GetListOfFriends().ls()

In [ ]:
tree, treeList, fileList = LoadTrees("cat performance.list",".*_qPt_tglDist","xxx","",0)
dfVariables=tree2Panda(tree,[".*Center",".*entries"],"entries>100", exclude=[".*his.*"])
dfCovar=tree2Panda(tree,[".*TRD.*CovarP4.*binMedian"],selection="entries>100",exclude=[".*ITS_.*"],
                   columnMask=[["_qPt_tglDist_binMedian",""], ["_hisCovar","_Cov"]])
dfCovar = pd.concat([dfCovar,dfVariables],axis=1)
print(dfCovar.shape)
dfCovar.head(5)
#dfCovar = dfCovar.sample(2000)
dfCovar["sentries"]=dfCovar.eval("sqrt(1/entries)")
dfCovar.head(5)

In [ ]:
output_file("performanceMap_covarP4.html")
figureArray = [
    [['qPtCenter'], ['TRDOff_CovP4_TRDv', 'TRDOn_CovP4_TRDv'], {"size": 2, "colorZvar":"tglCenter"}],
    [['qPtCenter'], ['TRDOn_CovP4_TRDv'], {"size": 2, "colorZvar":"tglCenter"}],
    [['qPtCenter'], ['TRDOn_CovP4_TRDv/TRDOff_CovP4_TRDv'], {"size": 3, "colorZvar":"tglCenter"}],
    ['table']
]

widgetParams=[
    ['range', ['TRDOff_CovP4_TRDv']],
    ['range', ['qPtCenter']],
    ['range', ['tglCenter']],
    ['range', ['sentries']],
]
tooltips = [("qPt", "@qPtCenter 1/GeV"), ("tgl", "@tglCenter"), ("entries", "@entries")]

figureLayoutDesc=[
    [0,1,{'commonY':1}],
    [2,{'plot_height':100}],
    {'plot_height':200,'sizing_mode':'scale_width','commonX':1}
]
widgetLayoutDesc=[ [0,1], [2,3], {'sizing_mode':'scale_width'} ]

fig=bokehDrawSA.fromArray(dfCovar, "entries>200", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,widgetLayout=widgetLayoutDesc,sizing_mode="scale_width")